In [1]:
import numpy as np
import pandas as pd
from data_loader import load_names_from_web, holdout_split

In [2]:
dfraw = load_names_from_web(category='national', hide_pre_1937=True, use_existing_files=True)
traintestval, holdout = holdout_split(dfraw)
trainval, test = holdout_split(traintestval)

In [39]:
trainval

,state,year,name,M/F,count
0,US,2000,Emily,F,25957
1,US,2000,Hannah,F,23085
2,US,2000,Ashley,F,17998
3,US,2000,Sarah,F,17712
4,US,2000,Alexis,F,17631
...,...,...,...,...,...
866154,US,2019,Zygmunt,M,5
866155,US,2019,Zyheem,M,5
866156,US,2019,Zyking,M,5
866157,US,2019,Zyn,M,5


In [44]:
def get_all_known_names(data):

    names = data.groupby(['state', 'name', 'M/F']).size().reset_index()
    names = names[['state', 'name', 'M/F']]
    return names

In [52]:
def evaluate(predictor, data_held_out, first_year_to_predict):

    most_recent_year = data_held_out['year'].max()
    
    years_to_predict = range(first_year_to_predict, most_recent_year+1)

    # only allow the model to see data from before the year to predict
    historical_data = data_held_out[data_held_out['year'] < first_year_to_predict]

    # get our model's predictions
    # predictions = predictor.predict(historical_data, years_to_predict)

    all_known_names = get_all_known_names(historical_data)

    for year_to_predict in years_to_predict:

        print(f'Predictions for {year_to_predict}:')

        names_to_predict = all_known_names.copy()
        names_to_predict['year'] = year_to_predict
        # display(names_to_predict)

        observed = names_to_predict.merge(data_held_out, how='left', on=['state', 'name', 'M/F', 'year'])

        # for now, fill in missing values with 0
        # we may want to change this to 2 in the original data, especially for names that have already appeared. but haven't figured out how to do that yet
        observed['count'] = observed['count'].fillna(0)
        observed = observed.rename(columns={'count': 'count_true'})
        # display(observed)

evaluate(None, test, 2018)

Predictions for 2018:
Predictions for 2019:
Predictions for 2020:
Predictions for 2021:
Predictions for 2022:


In [ ]:
class DummyPredictor